In [4]:
from Geometry import *
import numpy as np
#import lmfit as lf
from lmfit import Parameters, minimize
from scipy.linalg import sqrtm

def getAllDecendants(octTree):
    '''Get all lowest chilren on tree.'''
    out = []
    if octTree.hasChildren:
        for child in octTree.children:
            out = out + getAllDecendants(child)
        return out
    else:
        return [octTree]
    
def generateModelFromOctree(octTree,numRays,propName='Ne'):
    '''Generate '''
    voxels = getAllDecendants(octTree)        
    N = len(voxels)
    G = np.zeros([numRays,N])
    m = np.zeros(N)
    Cm = np.zeros(N)
    x = np.zeros([N,3])
    if propName not in voxels[0].properties.keys():
        #zero model if no property
        i = 0
        while i < N:
            vox = voxels[i]
            for j in vox.lineSegments.keys():
                G[j,i] = vox.lineSegments[j].sep   
            x[i,:] = vox.centroid
            i += 1
        return G,Cm,m,x
    i = 0
    while i < N:
        vox = voxels[i]
        for j in vox.lineSegments.keys():
            G[j,i] = vox.lineSegments[j].sep   
        m[i] = vox.properties[propName][1]
        Cm[i] = vox.properties[propName][2]
        x[i,:] = vox.centroid
        i += 1
    return G,Cm,m,x
        
def setOctTreeModel(octTree,m,Cm,propName='Ne',propType='extensive'):
    '''Set the model in the octTree. 
    Assumes the model is derived from the same octTree and
    Cm is the diagonal of the covariance.'''
    voxels = getAllDecendants(octTree)
    N = len(voxels)
    i = 0
    while i < N:
        vox = voxels[i]
        vox.properties[propName] = [propType,m[i],Cm[i]]
        vox.lineSegments = {}
        i += 1
    
def forwardProblem(rays,octTree):
    '''Perform the forward problem.'''
    
    pass
    
def noRefractionTECInversion(antennaLocs,antennaTEC, sourcePos):
    '''Invert the 3d TEC based on total measurments on the ground (after factor)
    antennaLocs are in uvw coords of shape
    antennaTEC are NxM'''
    pass

def makeRaysFromSourceAndReciever(recievers,sources):
    rays = []
    count = 0
    for r in recievers:
        for s in sources:
            rays.append(LineSegment(r,s,id=count))
            count += 1
    return rays

def LinearSolution(dobs,G,Cd,Cmprior,mprior):
    '''Assumes d = int(G * m)'''
    #forward problem
    print("Doing forward problem")
    #d = np.log(G.dot(np.exp(mprior)))
    d = G.dot(mprior)
    print("Calculating residuals:")
    residuals = dobs - d
    Gt = G.transpose()
    #smooth and adjoint
    print("Calculating smoothing matrix")
    smooth = np.linalg.inv(G.dot(Cmprior).dot(Gt) + Cd)
    #print(smooth)
    print("Calculating adjoint")
    adjoint = Cmprior.dot(Gt).dot(smooth)
    #print(adjoint)
    print("updating model")
    m = mprior + adjoint.dot(residuals)
    print("updating covariance")
    Cm = Cmprior - adjoint.dot(G).dot(Cmprior)
    return m,Cm  
    
def compute3dExponentialCovariance(sigma2,L,x):
    N = x.shape[0]
    C = np.zeros([N,N])
    i = 0
    while i < N:
        C[i,i] = sigma2
        j = i+1
        while j < N:
            d = x[i,:] - x[j,:]
            C[i,j] = sigma2*np.exp(-np.linalg.norm(d)/L)
            C[j,i] = C[i,j]
            j += 1
        i += 1
    C[C<epsFloat] = 0.
    return C

def ExampleLinearSolution():
    levels = 3
    fileName = "octTree_{0}levels.npy".format(levels)
    baseWidth = 1.
    height = 1000.
    try:
        octTree = loadOctTree(fileName)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    except:
        octTree = OctTree([0,0,height/2.],dx=baseWidth,dy=baseWidth,dz=height)
        subDivideToDepth(octTree,levels)
        saveOctTree(fileName,octTree)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
        
    
    mexact = np.zeros_like(m)
    M = 2
    print("Generating {0} component model".format(M))
    for i in range(M):
        loc = np.array([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = height/2.,high = height)])
        mexact += 0.3 + np.random.uniform()*np.exp(-np.sum((x - loc)**2,axis=1)/(height/8.)**2)
    modelVar = 0.1
    print("Computing model 3d exponential covariance")
    Cmexact = compute3dExponentialCovariance(modelVar,baseWidth/2.,x)
    print("Setting octTree with model.")
    setOctTreeModel(octTree,mexact,np.diag(Cmexact),propName='Ne',propType='extensive')
    M = 5
    print("Generating {0} recievers".format(M))
    recievers = []
    for i in range(M):
        recievers.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               -epsFloat])
    M = 10
    print("Generating {0} sources".format(M))
    sources = []
    for i in range(M):
        sources.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               height])
    rays = makeRaysFromSourceAndReciever(recievers,sources)
    print("Propagating {0} rays".format(len(rays)))
    for ray in rays:
        forwardRay(ray,octTree)
    print("Pulling ray propagations.")
    G,CmVar,mexact,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
    
    #generate model
    print("Computing fake observations")
    dobs = []
    for i in range(20):
        dobs.append(G.dot(np.random.multivariate_normal(mean=mexact, cov = Cmexact)))
        #print(dobs[-1])
    dobs = np.array(dobs)
    Cd = np.cov(dobs.transpose())
    dobs = np.mean(dobs,axis=0)
    #print (dobs)
    mprior = mexact/0.5
    print("Solving for model from rays:")
    print("solution with Cm exact input")
    m,Cm = LinearSolution(dobs,G,Cd,Cmexact,mprior)
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact,label='ex')
    plt.plot(mprior,label='pri')
    plt.legend(frameon=False)
    plt.show()
    plot3OctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m)
    f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
    ax1.imshow(Cm)
    ax2.imshow(Cmexact)
    plt.show()
    print("solution with Cm diagonal")
    m,Cm = LinearSolution(dobs,G,Cd,np.diag(np.diag(Cmexact)),mprior)
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact,label='ex')
    plt.plot(mprior,label='pri')
    plt.legend(frameon=False)
    plt.show()
    plotOctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m)
    f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
    ax1.imshow(Cm)
    ax2.imshow(Cmexact)
    plt.show()

def transformCov2Log(Cm_linear,mean_linear):
    '''Transform covariance matrix from linear model to log model using:
    cov(y1,y2) = <y1y2> - <y1><y2>
    with,
    y = log(x/K)
    thus,
    <y1y2> ~ y1y2 + 0.5*(var(x1)y1''y2 +var(x2)y2''y1) + cov(x1,x2)y1'y2' 
    = log(x1/K)log(x2/K) - 0.5*(var(x1)log(x2/K)/x1**2 +var(x2)log(x1/K)/x2**2) + cov(x1,x2)/x1/x2 
    and,
    <y1> ~ y1 + 0.5*var(x1)y1''
    = log(x1/K) - 0.5*var(x1)/x1**2
    Update using newer tecnique 
    '''
    Cm_log = np.log(1 + Cm_linear/np.outer(mean_linear,mean_linear))
    return Cm_log

def mcTransformCov2Log(m_linear,Cm_linear,K):
    
    res = []
    i = 0
    while i < 10:
        res.append(np.log(np.abs(np.random.multivariate_normal(mean=m_linear, cov = Cm_linear))/K))
        i += 1
    Cm_log = np.cov(np.transpose(res))
    print(Cm_log)
    m_log = np.mean(res,axis=0)
    return m_log,Cm_log

def mcTransformCov2Lin(m_log,Cm_log,K):
    
    res = []
    i = 0
    while i < 10:
        res.append(K*np.exp(np.random.multivariate_normal(mean=m_log, cov = Cm_log)))
        i += 1
    Cm_lin = np.cov(np.transpose(Cm_log))
    m_lin = np.mean(res,axis=0)
    return m_lin,Cm_lin

def transformCov2Linear(Cmprior_log,model_log,K):
    '''Transform covariance matrix from log model to ling model using:
    cov(x1,y2) = <y1y2> - <y1><y2>
    with,
    y = K*exp(x)
    thus,
    <y1y2> ~ y1y2 + 0.5*(var(x1)y1''y2 +var(x2)y2''y1) + cov(x1,x2)y1'y2' 
    = K*exp(x1)K*exp(x2)*(1 + 0.5*(var(x1) +var(x2)) + cov(x1,x2)) 
    and,
    <y1> ~ y1 + 0.5*var(x1)y1''
    = K*exp(x1)(1 + 0.5*var(x1))
    '''
    mn = K*np.exp(model_log)
    Cmprior = np.zeros_like(Cmprior_log)
    i = 0
    while i < mn.shape[0]:
        j = i
        while j < mn.shape[0]:
            Cmprior[i,j] = mn[i]*mn[j] * (1 + 0.5*(Cmprior_log[i,i] + Cmprior_log[j,j]) + Cmprior_log[i,j] - (1 + 0.5*Cmprior_log[i,i])*(1 + 0.5*Cmprior_log[j,j]))
            Cmprior[j,i] = Cmprior[i,j]
            j += 1
        i += 1
    return Cmprior
    
def SteepestDescent(octTree,rays,dobs,Cd,Cmprior,mprior):
    '''Assumes d = log(K*int(G * exp(m))) and that input is linear versions'''
    def updater(x,G):
        eps = np.zeros(x.shape[0])
        i = 0
        while i< x.shape[0]:
            if np.sum(G[:,i]) > 0:
                    eps[i] = 0.1
            else:
                eps[i] = 0.01
            i += 1
        return eps
    
    iter = 0
    mn = mprior
    Cmprior = Cmprior
    while iter < 10:
        #forward problem
        print("Setting octTree with model_{0}".format(iter))
        setOctTreeModel(octTree,mn,np.diag(Cmprior),propName='Ne',propType='extensive')
        print("Propagating {0} rays".format(len(rays)))
        for ray in rays:
            forwardRay(ray,octTree)
        print("Pulling ray propagations.")
        G,CmVar,mexact,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
        print("Doing forward problem")
        d = G.dot(mn)
        print("Calculating residuals, Sum:")
        residuals = d - dobs
        print(np.sum(residuals**2))
        #print(residuals.shape)
        print("Calculating weighting residuals")
        weightedRes = np.linalg.pinv(Cd).dot(residuals)
        print(Cd,weightedRes)
        #print(weightedRes,np.linalg.solve(Cd,residuals))
        #next part should be changed
        #Gt.Cd^-1.(d-dobs)
        Gt = G.transpose()
        #smooth and adjoint
        print("Calculating adjoint")
        dm = Cmprior.dot(Gt).dot(weightedRes)
        print("updating model")
        mn = mn - updater(x,G)*(dm + mn - mprior)
        iter += 1
    
    print("updating covariance")
    print("Calculating smoothing matrix")
    smooth = np.linalg.pinv(G.dot(Cmprior).dot(Gt) + Cd)
    print("Calculating adjoint")
    adjoint = Cmprior.dot(Gt).dot(smooth)
    Cm = Cmprior - adjoint.dot(G).dot(Cmprior)
    return mn,Cm

def ionosphereModel(x,dayTime=True,bump=False):
    h = x[2]
    Nf1 = 4*np.exp((h-300)/100.)/(1 + np.exp((h-300)/100.))**2
    if dayTime:
        Ne = 10**(-0.5)*4*np.exp((h-85.)/50.)/(1 + np.exp((h-85.)/50.))**2
        res = Nf1 + Ne
    else:
        res = Nf1
    if bump:
        res += 0.5*np.exp(-np.sum((x - np.array([50,50,400]))**2)/100.**2)
    return res

def constructIonosphereModel(maxBaseline):
    '''initialize with 1/m^3 at 300km +- 150km'''

    fileName = "ionosphereModel.npy"
    height = 1000.#km
    octTree = OctTree([0,0,height/2.],dx=maxBaseline,dy=maxBaseline,dz=height)
    #level 3 - all
    subDivide(subDivide(octTree))
    subDivide(octTree)
    voxels = getAllDecendants(octTree)
    voxels = []
    for vox in voxels:
        #level 4 - 250 to 750
        if (vox.centroid[2] > 250) and (vox.centroid[2] < 750):
            subDivide(vox)
        #level 5 - 250 to 500
        if (vox.centroid[2] > 250) and (vox.centroid[2] < 500):
            subDivide(vox)
    G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    i = 0
    while i < x.shape[0]:
        m[i] = ionosphereModel(x[i,:],dayTime=True,bump=True)
        i += 1
    setOctTreeModel(octTree,m,np.ones_like(m)*0.1,propName='Ne',propType='extensive')
    saveOctTree(fileName,octTree)
    #plotOctTreeXZ(octTree,ax=None)
    #plotOctTree3D(octTree,model=m)
    return octTree
    
def residuals(params_,G,x,nComp,dobs=None):
    '''params contain the model'''
    params = params_.valuesdict()
    N = x.shape[0]
    m = np.zeros(N)
    i = 0
    while i < nComp:
        loc = np.array([params["x{0}".format(i)],params["y{0}".format(i)],params["z{0}".format(i)]])
        d = x - loc
        m += params["a{0}".format(i)] * np.exp(-np.sum(d*d,axis=1) / params["s{0}".format(i)] )
        i += 1
    d = G.dot(m)
    if dobs is None:
        return d
    res = d - dobs
    print (np.sum(res))
    return res

def initHomogeneousModel(G,dobs):
    return np.sum(dobs)/np.sum(G)

def berrymanSol(G,dobs,mprior=None,mu = 0.0,Cd=None,Cm = None):
    '''Solve d=G.m minimizing misfit:
    (dobs-d)^t.W1.(dobs-d) + mu (m - mprior)^t.W2.(m-mprior)
    with the berryman choice of W1, W2.
    G is mxn, m - num rays, n - num cells'''
    m = G.shape[0]
    n = G.shape[1]

    
    if Cd is not None:
        L = Cd +  np.diag(np.sum(G,axis=1))
    else:
        #row sums, length of path i
        L = np.diag(np.sum(G,axis=1))
    if Cm is not None:
        C = np.linalg.pinv(Cm + np.diag(np.sum(G,axis=0)))
    else:
        #col sums, legnth of all rays in cell j (coverage)
        C = np.diag(np.sum(G,axis=0))  
    Linv = np.linalg.pinv(L)  
    Cinv = np.linalg.pinv(C)
    #m-vec choice
    u = np.ones(m)
    #n-vec
    v = Cinv.dot(G.transpose()).dot(u)
    #v = np.ones(n)
    sigma0 = u.transpose().dot(dobs)/(u.transpose().dot(L).dot(u))
    if mprior is None:
        #weight for mean background m0 = (u^t.L.W3.dobs/u^t.L.W3.L.u)v
        #W3 = inv(L)
        #W3 = Linv
        #mprior = u.transpose().dot(L).dot(W3).dot(dobs)/(u.transpose().dot(L).dot(W3).dot(L).dot(u))*v
        mprior = sigma0*v
    #W1 = Linv
    #D = np.sqrt(C)
    #A = np.sqrt(W1).dot(G).dot(inv(D))
    Linv12 = sqrtm(Linv)
    Cinv12 = sqrtm(Cinv)
    A = Linv12.dot(G).dot(Cinv12)
    AtA = A.transpose().dot(A)
    print("eigen val solve At.A",AtA)
    #sorted in ascending order
    sing,eigv = np.linalg.eigh(AtA)
    #Zj = xj^t.A^t.Linv12.dobs
    zb = sqrtm(C).dot(mprior)
    dz = np.zeros(n)
    
    adjoint = A.transpose().dot(Linv12).dot(dobs)
    i = len(sing) - 2
    while i >= 0:
        Zj = eigv[:,i].transpose().dot(adjoint)
        #print(Zj)
        
        if np.isnan(sing[i]) or sing[i] < 1e-5:
            print("rank: {0}".format(len(sing)-1-i))
            break
        
        dz += Zj*eigv[:,i]/(sing[i]+mu)
        i -= 1
    #compare with 
    #zcomp = np.linalg.pinv(AtA).dot(Cinv12).dot(G.transpose()).dot(Linv).dot(dobs)
    z = dz + zb
    m = Cinv12.dot(z)
    return np.abs(m)
    
def MetropolisSolution(G,dobs,Cd,Cmprior,mprior):
    postDist = []
    iter = 0
    T0 = 1.
    size = 1000
    Cdinv_ = np.linalg.pinv(Cd)
    mML = mprior
    Cm = Cmprior
    while iter < 100:
        print("Metropolis iter-{0}".format(iter))
        T = T0/(1 + iter)
        Cdinv = Cdinv_/T
        Cmsample = Cmprior*T
        count = 0
        mCandidate = np.copy(mML)
        d = (G.dot(mCandidate) - dobs)
        Li = np.exp(-d.transpose().dot(Cdinv).dot(d)/2.)
        while count < 100:
            print("New sample batch: {0}".format(count))
            #sample prior
            msample = np.abs(np.random.multivariate_normal(mean=mML, cov = Cmsample))
            # posterior distribution
            #forward problems
            i = 0
            while i < len(mML):
                mCandidate[i] = msample[i]
                d = (G.dot(mCandidate) - dobs)
                Lj = np.exp(-d.transpose().dot(Cdinv).dot(d)/2.)
                if Lj > Li:
                    Li = Lj
                    count += 1
                else:
                    if np.random.uniform() < Lj/Li:
                        Li = Lj
                        count += 1
                    else:
                        mCandidate[i] = mML[i]
                i += 1
        postDist.append(mCandidate)
        mML = mCandidate#np.mean(postDist,axis=0)
        iter += 1
    Cm = np.cov(postDist,rowvar=0)
    return mML,Cm
            
def LMSol(G,mprior,Cd,Cm,dobs,mu=1.,octTree=None):
    """Assume the frechet derivative is,
    G(x) = exp"""

    K = np.mean(mprior)
    mlog = np.log(mprior/K)
    
    Cm_log = np.log(1. + Cm/K**2)#transformCov2Log(Cm,mprior)
    #Cdinv = np.linalg.pinv(Cd)
    C = np.sum(G>0,axis=0)
    C = C/np.max(C)
    C[C==0] = np.min(C[C>0])/2.
    C = np.sum(G,axis=0)
    C = C/np.max(C)
    res = 1
    iter = 0
    while res > 1e-7:
        #forward transform
        mForward = K*np.exp(mlog)
        g = G.dot(mForward)
        J = G*mForward
        #residuals g - dobs -> -dm
        res = g - dobs
        
        #A1 = J.transpose().dot(Cdinv)
        #Cmlog_inv = A1.dot(J) + mu*Cm_log
        #dm,resi,rank,s = np.linalg.lstsq(Cmlog_inv,A1.dot(res))
        #S = mu Cd + J.Cm.J^t
        #S = int Ri Rj k^2 exp(m(x) + m(x')) sigma^2 exp(-|x-x'|/L) + Cd
        
        #K int dV Cm(x,x') J(x') del(i)
        P1 = Cm_log.dot(J.transpose())
        smooth = np.linalg.pinv(mu*Cd + J.dot(P1))
        dm = P1.dot(smooth).dot(res)
        res = np.sum(dm**2)
        print("Iter-{0} res: {1}".format(iter,res))
        #converage learn propto length of rays in cells
        
        mlog -= dm*C
        iter += 1
    
    return K*np.exp(mlog), np.exp(Cm_log)

def FunctionalSol(G,mprior,Cd,dobs,x,L=20.,sigma2=0.01,mu=1.):
    """Assume the frechet derivative is,
    G(x) = exp"""

    #transform to log model so that g = K int ds exp(m)
    K = np.mean(mprior)
    mlog = np.log(mprior/K)
    #Cm_log = transformCov2Log(Cm,mprior)
    N = G.shape[0]
    M = G.shape[1]
    
    iter = 0
    while iter < 100:
        #forward transform
        mlin = K*np.exp(mlog)
        g = G.dot(mlin)
        #residuals g - dobs -> -dm
        res = g - dobs
        S = np.copy(Cd)
        i = 0
        while i < N:#for ith ray
            j = 0
            while j < N:#for jth ray
                c = 0
                while c < M:#for cell c in ray i
                    dx = x - x[c,:]#Nx3
                    xabs = np.sqrt(np.diag(dx.dot(dx.transpose())))
                    Cmlog = np.log(1. + sigma2*np.exp(-xabs/L)/K**2)
                    #print(Cmlog.shape,dx.shape)
                    S[i,j] += G[i,c]*mlin[c]*(G[j,:]*mlin).dot(Cmlog)
                    c += 1
                j += 1
            i += 1
        T = np.linalg.pinv(S)
        #weighted res
        weightedRes = T.dot(res)
        dm = np.zeros_like(mlog)
        alpha = np.zeros_like(dm)
        i = 0
        while i < N:#for each ray
            alpha *= 0#zero alpha
            c = 0
            while c < M:#integral loop
                dx = x - x[c,:]#Nx3
                xabs = np.sqrt(np.diag(dx.dot(dx.transpose())))
                Cmlog = np.log(1. + sigma2*np.exp(-xabs/L)/K**2)
                alpha += G[i,c]*mlin[c]*Cmlog
                c += 1
            dm += weightedRes[i]*alpha
            i += 1
        print (np.sum(dm**2))
        #converage learn propto length of rays in cells
        mlog -= dm
        iter += 1
    #convergence mlog
    
    return K*np.exp(mlog), None

def gradientCheck(mprior,G):
    eps = 7./4. - 3./4. - 1.
    eps = epsFloat
    N = np.size(mprior)
    M = G.shape[0]
    K = np.mean(mprior)
    mlog = np.log(mprior/K)
    mForward = K*np.exp(mlog)
    g0 = G.dot(mForward)
    J = G*mForward
    Jexact = np.zeros([M,N])
    i = 0
    while i < N:
        mlog_old = mlog[i]
        mlog[i] += eps
        mForward = K*np.exp(mlog)
        g = G.dot(mForward)
        Jexact[:,i] = (g - g0)/eps
        #print(Jexact[:,i])
        mlog[i] = mlog_old
        i += 1
    import pylab as plt
    plt.imshow(J-Jexact)
    plt.colorbar()
    plt.show()
    return J,Jexact
        
if __name__=='__main__':
    print("Constructing ionosphere model")
    maxBaseline = 100
    octTree = constructIonosphereModel(maxBaseline)
    M = 40
    print("Generating {0} recievers".format(M))
    recievers = []
    for i in range(M):
        recievers.append([np.random.uniform(low = -maxBaseline/2.,high = maxBaseline/2.),
               np.random.uniform(low = -maxBaseline/2.,high = maxBaseline/2.),
               -epsFloat])
    M = 15
    print("Generating {0} sources".format(M))
    sources = []
    for i in range(M):
        sources.append([np.random.uniform(low = -maxBaseline/2.,high =maxBaseline/2.),
               np.random.uniform(low = -maxBaseline/2.,high = maxBaseline/2.),
               1000.])
    rays = makeRaysFromSourceAndReciever(recievers,sources)
    print("Propagating {0} rays".format(len(rays)))
    for ray in rays:
        forwardRay(ray,octTree)
    print("Pulling ray propagations.")
    G,mVar_lin,mexact_lin,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
    mprior = []
    i = 0
    while i < x.shape[0]:
        mprior.append(ionosphereModel(x[i,:],dayTime=False))
        i += 1
    mprior = np.array(mprior)
    #J,Jexact = gradientCheck(mprior,G)
    
    #ExampleLinearSolution()
    modelVar = np.mean(mVar_lin)
    print("Computing model 3d exponential covariance")
    Cmexact_lin = compute3dExponentialCovariance(0.1**2,20.,x)
    #K = np.mean(mexact_lin)
    #mexact_log,Cmexact_log = mcTransformCov2Log(mexact_lin,Cmexact_lin,K)
    #generate model
    print("Computing fake observations")
    #mexact_lin = np.abs(np.random.multivariate_normal(mean=mexact_lin, cov = Cmexact_lin))
    dobs = []
    for i in range(10):
        #dobs.append(np.log(G.dot(K*np.exp(np.random.multivariate_normal(mean=mexact_log, cov = Cmexact_log)))))
        dobs.append(G.dot(np.abs(np.random.multivariate_normal(mean=mexact_lin, cov = Cmexact_lin))))
        #print(dobs[-1])
    dobs = np.array(dobs)
    Cd = np.cov(dobs.transpose())
    #print(dobs)
    dobs = G.dot(mexact_lin)
    #dobs = G.dot(mexact_lin)
    #print (dobs)
    #mprior = np.ones_like(mexact_lin)*initHomogeneousModel(G,dobs)
    #mprior = np.random.multivariate_normal(mean=mexact_lin, cov = Cmexact_lin)
    Cmprior = Cmexact_lin
    #row sums, length of path i
    L = np.diag(np.sum(G,axis=1))
    Linv = np.linalg.pinv(L)
    #col sums, legnth of all rays in cell j (coverage)
    C = np.diag(np.sum(G,axis=0))
    Cinv = np.linalg.pinv(C)
    Cinv = Cinv/np.max(Cinv)*0.1
    #Cmprior = Cinv.dot(Cmprior)
    #Cd = Cd.dot(L)
    #print(C,L,Cinv,Linv)
    print("Solving for model from rays:")
    #m,Cm = LinearSolution(dobs,G,Cd,Cmprior,mprior)
    m,Cm = LMSol(G,mprior,Cd,Cmprior,dobs,mu=1.0)
    #m,Cm = FunctionalSol(G,mprior,Cd,dobs,x,L=20.,sigma2=0.01,mu=1.)
    #m,Cm = MetropolisSolution(G,dobs,Cd,Cmprior,mprior)
    #m = berrymanSol(G,dobs,mprior=None,Cd=Cd,Cm=None,mu=0.00)
    #for i in range(20):
    #    m = berrymanSol(G,dobs,mprior=m,Cd=Cd,Cm=None,mu=0.00001)
    #Cm = Cmprior
    #m,Cm = SteepestDescent(octTree,rays,dobs,Cd,Cmprior,mprior)
    #params = vec2Parameters(mprior)
    #params = InitParameters(5)
    #res = minimize(residuals, params, args=(G,x,5), kws={"dobs":dobs})
    #print (res)
    #m,Cm = LinearSolution(dobs,G,Cd,Cmprior,mprior)
    CmCm = Cm.dot(np.linalg.inv(Cmprior))
    R = np.eye(CmCm.shape[0]) - CmCm
    print("Resolved by dataSet:{0}, resolved by a priori:{1}".format(np.trace(R),np.trace(CmCm)))
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact_lin,label='ex')
    plt.plot(mprior,label='pri')
    C = np.sum(G>0,axis=0)
    C = C < 3
    plt.scatter(np.arange(len(m))[C],m[C])
    
    plt.legend(frameon=False)
    plt.show()
    plotOctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m,rays=False)
   

Constructing ionosphere model
Generating 40 recievers
Generating 15 sources
Propagating 600 rays
Pulling ray propagations.
Computing model 3d exponential covariance
Computing fake observations
Solving for model from rays:
Iter-0 res: 21.801256143639186
Iter-1 res: 11.962712414457828
Iter-2 res: 8.055321657970346
Iter-3 res: 5.913424162803477
Iter-4 res: 4.5733235738851565
Iter-5 res: 3.664373163407469
Iter-6 res: 3.0122503292556018
Iter-7 res: 2.5245199065776966
Iter-8 res: 2.147882428228371
Iter-9 res: 1.8495857067427053
Iter-10 res: 1.6084636434437745
Iter-11 res: 1.4102659860139486
Iter-12 res: 1.2450672347585703
Iter-13 res: 1.1057527735644332
Iter-14 res: 0.9870952456794051
Iter-15 res: 0.8851701036456757
Iter-16 res: 0.7969741527502036
Iter-17 res: 0.7201698873348165
Iter-18 res: 0.6529101362237133
Iter-19 res: 0.5937153068432172
Iter-20 res: 0.5413858512017832
Iter-21 res: 0.49493877937698977
Iter-22 res: 0.453560878677664
Iter-23 res: 0.4165737232042469
Iter-24 res: 0.383407128

In [2]:
f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
ax1.imshow(Cm)
ax2.imshow(Cmexact_lin)
plt.show()



In [10]:
import pylab as plt
plt.hist(np.sum(G>0,axis=0),bins=100)
plt.show()


In [12]:
np.median(np.sum(G>0,axis=0))

2.0